# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list[0:20])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) 
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### 1. Modeling First query 
`SELECT artist, song, length FROM artist_song_session WHERE sessionid = '338' AND itemInSession = '4' `

##### 1.1 Create `artist_song_session` table

In [8]:
query = "DROP TABLE IF EXISTS artist_song_session"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS artist_song_session"
query = query + "(sessionid VARCHAR, itemInSession VARCHAR, artist TEXT, song TEXT, length VARCHAR, PRIMARY KEY(sessionid, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

##### 1.2 Insert data into the table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    csvreader
    for line in csvreader:
        query = "INSERT INTO artist_song_session(sessionid, itemInSession, artist, song, length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (line[8], line[3],line[0], line[9], line[5]))

##### 1.3 Check the query result

In [10]:
query = "SELECT artist, song, length FROM artist_song_session WHERE sessionid = '338' AND itemInSession = '4'"

try : 
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.length, row.song)

Faithless 495.3073 Music Matters (Mark Knight Dub)


#### 2. Modeling Second Query 
`SELECT artist, song, firstName, lastName FROM song_playlist WHERE sessionid = 338 AND itemInSession = 4`

In [11]:
query = "DROP TABLE IF EXISTS song_playlist"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS song_playlist"
query = query + "(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, \
                PRIMARY KEY((userid, sessionid),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        try :
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e : 
            print(e)
            break

In [ ]:
query = "SELECT artist, song, firstname, lastname FROM song_playlist WHERE userid = 10 AND sessionid = 182"
try : 
    rows = session.execute(query)
except Exception as e:
    print(e)

try:
    for row in rows[0:10]:
        print(row[0:])
except Exception as e :
    print(e)

#### 3. Modeling Third Query 
`SELECT firstName, lastName FROM user_name_session WHERE song = 'All Hands Against His Own`

In [ ]:
query = "DROP TABLE IF EXISTS user_name_session"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS user_name_session"
query = query + "(song VARCHAR, userid VARCHAR, firstName VARCHAR, lastName VARCHAR, PRIMARY KEY(song, userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_name_session(song, userid, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9], line[10], line[1],line[4]))

query = "SELECT firstName, lastName FROM user_name_session WHERE song = 'All Hands Against His Own'"
try : 
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows[0:10]:
    print(row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE artist_song_session")
    session.execute("DROP TABLE song_playlist")
    session.execute("DROP TABLE user_name_session")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()